Import Necessary Libraries


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV  # Ensure this line is present and correct
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


Load the Dataset

In [2]:
df = pd.read_csv('credit_risk_dataset.csv')


Explore the Dataset

In [3]:
print(df.head())


   person_age  person_income person_home_ownership  person_emp_length  \
0          22          59000                  RENT              123.0   
1          21           9600                   OWN                5.0   
2          25           9600              MORTGAGE                1.0   
3          23          65500                  RENT                4.0   
4          24          54400                  RENT                8.0   

  loan_intent loan_grade  loan_amnt  loan_int_rate  loan_status  \
0    PERSONAL          D      35000          16.02            1   
1   EDUCATION          B       1000          11.14            0   
2     MEDICAL          C       5500          12.87            1   
3     MEDICAL          C      35000          15.23            1   
4     MEDICAL          C      35000          14.27            1   

   loan_percent_income cb_person_default_on_file  cb_person_cred_hist_length  
0                 0.59                         Y                           3  


Handle Missing Values

In [4]:
print(df.isnull().sum())  # Check for missing values
df = df.dropna()  # Drop missing values or use imputation as appropriate


person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64


Encode Categorical Features

In [5]:
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade'], drop_first=True)


In [6]:
df['cb_person_default_on_file'] = df['cb_person_default_on_file'].map({'Y': 1, 'N': 0})


In [7]:
print(df['cb_person_default_on_file'].unique())  # It should print [1 0]


[1 0]


Split Data

In [8]:
from sklearn.model_selection import train_test_split

X = df.drop('loan_status', axis=1)
y = df['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
# Set up parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']}

# Create a GridSearchCV object
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

# Print best parameters
print(grid.best_params_)

# Predict on the test data
grid_predictions = grid.predict(X_test)

# Print classification report
print(classification_report(y_test, grid_predictions))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  39.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  38.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  38.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  38.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  37.6s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  33.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  34.5s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  34.6s
[CV] END .......................C=0.1, ga

KeyboardInterrupt: 

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

param_dist = {'C': np.logspace(-3, 2, 6), 'gamma': np.logspace(-3, 2, 6), 'kernel': ['rbf']}

# Using fewer iterations and all CPU cores
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=5, verbose=2, n_jobs=-1)
random_search.fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Inte

RandomizedSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                                        'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                                        'kernel': ['rbf']},
                   verbose=2)

Normalize Data

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Create a Sequential model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


Build the Model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Train the Model

In [22]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
716/716 [==============================] - 1s 918us/step - loss: 0.3401 - accuracy: 0.8718 - val_loss: 0.2974 - val_accuracy: 0.8895
Epoch 2/50
716/716 [==============================] - 1s 780us/step - loss: 0.2810 - accuracy: 0.9016 - val_loss: 0.2886 - val_accuracy: 0.8987
Epoch 3/50
716/716 [==============================] - 1s 789us/step - loss: 0.2627 - accuracy: 0.9099 - val_loss: 0.2697 - val_accuracy: 0.9019
Epoch 4/50
716/716 [==============================] - 1s 835us/step - loss: 0.2533 - accuracy: 0.9141 - val_loss: 0.2592 - val_accuracy: 0.9076
Epoch 5/50
716/716 [==============================] - 1s 790us/step - loss: 0.2467 - accuracy: 0.9165 - val_loss: 0.2559 - val_accuracy: 0.9092
Epoch 6/50
716/716 [==============================] - 1s 762us/step - loss: 0.2408 - accuracy: 0.9181 - val_loss: 0.2535 - val_accuracy: 0.9104
Epoch 7/50
716/716 [==============================] - 1s 768us/step - loss: 0.2360 - accuracy: 0.9203 - val_loss: 0.2484 - val_accuracy:

Evaluate the Model

In [23]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

179/179 [==============================] - 0s 473us/step - loss: 0.2360 - accuracy: 0.9223
Accuracy: 92.23%


Confusion Matrix and Classification Report

In [24]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predicting the Test set results
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # convert probabilities to binary output

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification Report
cr = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(cr)


179/179 [==============================] - 0s 415us/step
Confusion Matrix:
[[4382   61]
 [ 384  901]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      4443
           1       0.94      0.70      0.80      1285

    accuracy                           0.92      5728
   macro avg       0.93      0.84      0.88      5728
weighted avg       0.92      0.92      0.92      5728

